# Compare predictions

In [27]:
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
OUTPUT_PATH = Path(".").absolute().parent / "output"

In [58]:
prediction_files = [
    OUTPUT_PATH / "inference_FurnitureVGG16BN_20180412_0719" / "predictions.csv",
    OUTPUT_PATH / "inference_FurnitureNASNetALarge_20180418_0635" / "predictions.csv",
    OUTPUT_PATH / "inference_FurnitureSqueezeNet350_20180415_1430" / "predictions.csv",
    OUTPUT_PATH / "inference_FurnitureInceptionV4_350_20180419_0623" / "predictions.csv",
]

In [59]:
names = [f.parent.name.replace("inference_", "") for f in prediction_files]
names

['FurnitureVGG16BN_20180412_0719',
 'FurnitureNASNetALarge_20180418_0635',
 'FurnitureSqueezeNet350_20180415_1430',
 'FurnitureInceptionV4_350_20180419_0623']

In [60]:
dfs = [pd.read_csv(f, index_col='id') for f in prediction_files]

In [61]:
merged_df = pd.concat([df for df in dfs], axis=1, names=['a', 'b', 'c', 'd'])
merged_df.columns = names

In [62]:
merged_df.head(10)

,FurnitureVGG16BN_20180412_0719,FurnitureNASNetALarge_20180418_0635,FurnitureSqueezeNet350_20180415_1430,FurnitureInceptionV4_350_20180419_0623
id,,,,
1,12,12,12,12
2,71,71,71,71
3,91,91,91,91
4,54,54,54,54
5,42,126,42,126
6,76,76,76,76
7,94,94,96,94
8,42,8,42,8
9,127,127,127,127


In [63]:
def get_decision_fn(weights):
    def fn(row):
        votes = np.zeros(128 + 1, dtype=np.int)
        for r, w in zip(row, weights):
            votes[r] += w
        return np.argmax(votes)
    return fn

In [66]:
merged_df.loc[:, 'MajVote'] = merged_df.apply(get_decision_fn(weights=[1.0, 3.0, 1.0, 2.5]), axis=1)

In [67]:
merged_df.head(10)

,FurnitureVGG16BN_20180412_0719,FurnitureNASNetALarge_20180418_0635,FurnitureSqueezeNet350_20180415_1430,FurnitureInceptionV4_350_20180419_0623,MajVote
id,,,,,
1,12,12,12,12,12
2,71,71,71,71,71
3,91,91,91,91,91
4,54,54,54,54,54
5,42,126,42,126,126
6,76,76,76,76,76
7,94,94,96,94,94
8,42,8,42,8,8
9,127,127,127,127,127


In [68]:
merged_df.tail(10)

,FurnitureVGG16BN_20180412_0719,FurnitureNASNetALarge_20180418_0635,FurnitureSqueezeNet350_20180415_1430,FurnitureInceptionV4_350_20180419_0623,MajVote
id,,,,,
12791,112,27,112,112,112
12792,82,82,82,82,82
12793,50,50,50,50,50
12794,111,111,111,111,111
12795,88,2,88,2,2
12796,77,77,77,77,77
12797,75,75,75,75,75
12798,58,3,58,58,58
12799,122,1,1,1,1


In [69]:
merged_df['MajVote'].to_csv("maj_votes_vgg_nasnet_sqnet_incv4.csv", header=["predicted"])

In [71]:
!head maj_votes_vgg_nasnet_sqnet_incv4.csv

id,predicted
1,12
2,71
3,91
4,54
5,126
6,76
7,94
8,8
9,127
